In [1]:
pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

class evaluation_model():
    def __init__(self):
        #modify the path to your own path
        checkpoint = torch.load('/kaggle/input/lab6file/file/checkpoint.pth')
        self.resnet18 = models.resnet18(pretrained=False)
        self.resnet18.fc = nn.Sequential(
            nn.Linear(512,24),
            nn.Sigmoid()
        )
        self.resnet18.load_state_dict(checkpoint['model'])
        self.resnet18 = self.resnet18.cuda()
        self.resnet18.eval()
        self.classnum = 24
    def compute_acc(self, out, onehot_labels):
        batch_size = out.size(0)
        acc = 0
        total = 0
        for i in range(batch_size):
            k = int(onehot_labels[i].sum().item())
            total += k
            outv, outi = out[i].topk(k)
            lv, li = onehot_labels[i].topk(k)
            for j in outi:
                if j in li:
                    acc += 1
        return acc / total
    def eval(self, images, labels):
        with torch.no_grad():
            #your image shape should be (batch, 3, 64, 64)
            out = self.resnet18(images)
            acc = self.compute_acc(out.cpu(), labels.cpu())
            return acc

In [3]:
import os

# 定义要创建的目录路径
directory_path = '/kaggle/working/weights'

# 使用os.makedirs()函数创建目录
os.makedirs(directory_path, exist_ok=True)

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import json
import os
from torchvision import transforms


with open("/kaggle/input/lab6file/file/objects.json" , 'r') as f:
    labels_mapping = json.load(f)

def getData(data_path):
    with open(data_path , 'r') as f:
        data = json.load(f)
    return data



class iclevrDataSet(Dataset):
    def __init__(self,root,mode):
        self.root = root
        self.labels_mapping = labels_mapping
        self.mode = mode
        if(mode == "train"):
            self.data = getData("/kaggle/input/lab6file/file/train.json")
            self.transforms = transforms.Compose([
                transforms.Resize([64, 64]),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
            ])
            self.image_names = list(self.data.keys())
        elif(mode == "test" or mode == "new_test"):
            self.data = getData("/kaggle/input/lab6file/file/"+mode+".json")
            #print(len(self.data))
        else : 
            raise ValueError(f"No {mode} mode!")
        

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        if(self.mode == "train"):
            img_name = self.image_names[index]
            img_labels = self.data[img_name] #image data labels 
            img_path = os.path.join(self.root,img_name)
            image = Image.open(img_path).convert('RGB')
            image = self.transforms(image)
            one_hot_label = np.zeros(len(self.labels_mapping), dtype=np.float32)
            #print(image.shape)
            for label in img_labels:
                one_hot_label[self.labels_mapping[label]] = 1

        elif(self.mode == "test" or self.mode == "new_test"):
            image = torch.randn(3, 64, 64)
            labels = self.data[index]
            one_hot_label = np.zeros(len(self.labels_mapping), dtype=np.float32)
            for label in labels:
                one_hot_label[self.labels_mapping[label]] = 1
       
        return image, torch.tensor(one_hot_label)

#test = iclevrDataSet("iclevr","new_test")
# a,b= test.__getitem__(3)
# print(b)

In [10]:
#from dataloader import iclevrDataSet
#from evaluator import evaluation_model
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
from diffusers import DDIMScheduler, DDPMPipeline
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers.optimization import get_cosine_schedule_with_warmup
from transformers import CLIPFeatureExtractor, CLIPTextModel
from diffusers import UNet2DModel,UNet2DConditionModel, DDPMScheduler
from accelerate import Accelerator
import argparse
from diffusers.optimization import get_cosine_schedule_with_warmup
net = UNet2DModel(
    sample_size=64,       # the target image resolution
    in_channels=3,                 # additional input channels for class condition
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=(
        "DownBlock2D",          # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",      # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",
        "AttnUpBlock2D",        # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",            # a regular ResNet upsampling block
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
    class_embed_type = None,
)
net.class_embedding = nn.Linear(24 ,512)

def tqdm_bar(pbar, loss, lr,epoch):
    pbar.set_description(f"(Epoch {epoch}, lr:{lr}", refresh=False)
    pbar.set_postfix(loss=float(loss), refresh=False)
    pbar.refresh()
def sample(net,noise_scheduler,dataloader):
    # Sampling loop
    for  img, cond in tqdm(dataloader, ncols=120):
        img = img.to("cuda")
        cond = cond.to("cuda")
        for t in(noise_scheduler.timesteps):
            # Get model pred
            with torch.no_grad():
                residual = net(img, t.to("cuda"), cond).sample  
            # Update sample with step
            img = noise_scheduler.step(residual, t, img).prev_sample
    return img

if __name__ == '__main__':

   # noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2')
    noise_scheduler = DDIMScheduler.from_pretrained("google/ddpm-cifar10-32")
    noise_scheduler.set_timesteps(num_inference_steps=50)
    noise_scheduler.config.clip_sample = False
    dataset = iclevrDataSet("/kaggle/input/lab6file/file/iclevr","train")
    train_loader = DataLoader(dataset,
                            batch_size=32,
                            num_workers=4,
                            shuffle=True)  
    
    dataset = iclevrDataSet("/kaggle/input/lab6file/file/iclevr","test")
    test_loader = DataLoader(dataset,
                            batch_size=32,
                            num_workers=4,
                            shuffle=False)  
    # How many runs through the data should we do?
    n_epochs = 150


    # Our network 

    #checkpoint = torch.load("/kaggle/working/weights/epoch=3.ckpt")
    eval_model = evaluation_model()
    ######################################################################################
    # Our loss finction
    loss_fn = nn.MSELoss()
    # The optimizer
    lr = 0.0001
    opt = torch.optim.AdamW(net.parameters(), lr=lr) 
    total_step = len(train_loader.dataset)// 32* n_epochs if len(train_loader.dataset) % 32 ==0 else (len(train_loader.dataset)// 33)* n_epochs

    lr_scheduler = get_cosine_schedule_with_warmup(
        optimizer=opt,
        num_warmup_steps=500,
        num_training_steps=(len(train_loader) * n_epochs),
    )
    # Keeping a record of the losses for later viewing

    # The training loop
    accelerator = Accelerator(
        mixed_precision="fp16",
        gradient_accumulation_steps=2,
    )
    if accelerator.is_main_process:
        accelerator.init_trackers("train_example")

    net,  opt, train_loader, lr_scheduler = accelerator.prepare(
        net,  opt, train_loader, lr_scheduler
    )
    ###########################################################################
    denormalize = transforms.Compose([
        transforms.Normalize(mean=(-1, -1, -1), std=(2, 2, 2))  
    ])
    
    best_acc = 0
    for epoch in range(n_epochs):
        total_loss = 0
        for  img, cond in (pbar := tqdm(train_loader, ncols=120)):
            opt.zero_grad()
            # Get some data and prepare the corrupted version
            img = img.to("cuda") 
            cond = cond.to("cuda")
            noise = torch.randn_like(img)
   
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (img.shape[0],)).long().to("cuda")

            #timesteps = torch.randint(0, 1000, (img.shape[0],)).long().to("cuda")
            noisy_x = noise_scheduler.add_noise(img, noise, timesteps)
            with accelerator.accumulate(net):
                # Get the model prediction
                pred = net(noisy_x, timesteps, cond).sample # Note that we pass in the labels y
                # Calculate the loss
                loss = loss_fn(pred, noise) # How close is the output to the noise
                # Backprop and update the params:
                accelerator.backward(loss)
                #accelerator.clip_grad_norm_(net.parameters(), 1.0)
                lr_scheduler.step()
                opt.step()
                opt.zero_grad()
                # Store the loss for later
                total_loss += loss.item() * img.size(0)
                tqdm_bar(pbar, loss, '{:.0e}'.format(lr_scheduler.get_lr()[0]),epoch)
        
        epoch_loss = total_loss/len(train_loader.dataset)
        decode_result = sample(net,noise_scheduler,test_loader)
        data_iterator = iter(test_loader)
        batch = next(data_iterator)
        _, labels = batch
        acc = eval_model.eval(decode_result,labels)
        print(f'Finished epoch {epoch}. Average of loss values: {epoch_loss:0.5f}')
        print(f'Test acc is {acc}')
        if(acc > best_acc):
            torch.save(net,os.path.join("/kaggle/working/weights","best.ckpt"))
            print(">>New best checkpoint save")
            best_acc = acc
        if((epoch+1)%10 == 0 or (epoch+1)==n_epochs):
            torch.save(net,os.path.join("/kaggle/working/weights", f"epoch={epoch}.ckpt"))
        grid_of_images = make_grid(denormalize(decode_result), nrow=8) 
        save_image(grid_of_images, "/kaggle/working/"+str(epoch)+'image.png')
        

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.55s/it]


Finished epoch 0. Average of loss values: 0.24551
Test acc is 0.1388888888888889
>>New best checkpoint save


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.54s/it]


Finished epoch 1. Average of loss values: 0.01697
Test acc is 0.09722222222222222


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.55s/it]


...

Finished epoch 108. Average of loss values: 0.00089
Test acc is 0.7777777777777778


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.61s/it]


Finished epoch 109. Average of loss values: 0.00091
Test acc is 0.7361111111111112


(Epoch 110, lr:7e-05:  74%|████████████████████████████████▋           | 419/563 [02:35<00:53,  2.70it/s, loss=0.000657]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:140                                                                                  │
│                                                                                                  │
│   137 │   │   │   │   # Calculate the loss                                                       │
│   138 │   │   │   │   loss = loss_fn(pred, noise) # How close is the output to the noise         │
│   139 │   │   │   │   # Backprop and update the params:                                          │
│ ❱ 140 │   │   │   │   accelerator.backward(loss)                                                 │
│   141 │   │   │   │   #accelerator.clip_grad_norm_(net.parameters(), 1.0)                        │
│   142 │   │   │   │   lr_scheduler.step()                                                        │
│   143 │   │   │   │   opt.step()                                                                 │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:1819 in backward               │
│                                                                                                  │
│   1816 │   │   elif self.distributed_type == DistributedType.MEGATRON_LM:                        │
│   1817 │   │   │   return                                                                        │
│   1818 │   │   elif self.scaler is not None:                                                     │
│ ❱ 1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                                                             │
│   1821 │   │   │   loss.backward(**kwargs)                                                       │
│   1822                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:200 in backward               │
│                                                                                                  │
│   197 │   # The reason we repeat same the comment below is that                                  │
│   198 │   # some Python versions print out the first line of a multi-line function               │
│   199 │   # calls in the traceback and some print out the last line                              │
│ ❱ 200 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   201 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   202 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   203                                                      

In [5]:
def sample(net,noise_scheduler,dataloader):

    # Sampling loop
    for  img, cond in tqdm(dataloader, ncols=120):
        img = img.to("cuda")
        cond = cond.to("cuda")
        for t in(noise_scheduler.timesteps):
            # Get model pred
            with torch.no_grad():
                residual = net(img, t.to("cuda"), cond).sample  # Again, note that we pass in our labels y
            # Update sample with step
            img = noise_scheduler.step(residual, t, img).prev_sample

    return img

In [12]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
from diffusers import DDIMScheduler, DDPMPipeline
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers.optimization import get_cosine_schedule_with_warmup
from transformers import CLIPFeatureExtractor, CLIPTextModel
from diffusers import UNet2DModel,UNet2DConditionModel, DDPMScheduler
from accelerate import Accelerator
import argparse
from diffusers.optimization import get_cosine_schedule_with_warmup
dataset = iclevrDataSet("/kaggle/input/lab6file/file/iclevr","new_test")
new_test_loader = DataLoader(dataset,
                            batch_size=32,
                            num_workers=4,
                            shuffle=False)  

#noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2')
noise_scheduler = DDIMScheduler.from_pretrained("google/ddpm-cifar10-32")
noise_scheduler.set_timesteps(num_inference_steps=50)
noise_scheduler.config.clip_sample = False
net = UNet2DModel().to("cuda")
eval_model = evaluation_model()
denormalize = transforms.Compose([
    transforms.Normalize(mean=(-1, -1, -1), std=(2, 2, 2))  
])
#checkpoint = torch.load("/kaggle/working/weights/best.ckpt")
weight_path = "/kaggle/input/lab6-best-weight/best.ckpt"
net = torch.load(weight_path)
decode_result = sample(net,noise_scheduler,new_test_loader)
data_iterator = iter(new_test_loader)
batch = next(data_iterator)
_, labels = batch
acc = eval_model.eval(decode_result,labels)
print(f'New Test acc is {acc}')
grid_of_images = make_grid(denormalize(decode_result), nrow=8) 
save_image(grid_of_images, "/kaggle/working/"+'new_test_image.png')

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.46s/it]


New Test acc is 0.8571428571428571


In [24]:
#生成每個time step 的圖片
def sample_and_save(net, noise_scheduler, dataloader, save_interval=50, save_path="/kaggle/working/"):
    counter = 0  # 初始化計數器
    for img, cond in tqdm(dataloader, ncols=120):
        img = img.to("cuda")
        cond = cond.to("cuda")
        for t in noise_scheduler.timesteps:
            # Get model pred
            with torch.no_grad():
                residual = net(img, t.to("cuda"), cond).sample

            # Update sample with step
            img = noise_scheduler.step(residual, t, img).prev_sample

            # 檢查是否達到保存圖像的計數器值
            if (counter % save_interval == 0) or counter == 999:
                grid_of_images = make_grid(denormalize(img), nrow=8)
                filename = f"image_timestep={counter}.png"
                save_image(grid_of_images, os.path.join(save_path, filename))               
            counter += 1
    return img


In [25]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
from diffusers import DDIMScheduler, DDPMPipeline
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers.optimization import get_cosine_schedule_with_warmup
from transformers import CLIPFeatureExtractor, CLIPTextModel
from diffusers import UNet2DModel,UNet2DConditionModel, DDPMScheduler
from accelerate import Accelerator
import argparse
from diffusers.optimization import get_cosine_schedule_with_warmup
dataset = iclevrDataSet("/kaggle/input/lab6file/file/iclevr","test")
new_test_loader = DataLoader(dataset,
                            batch_size=32,
                            num_workers=4,
                            shuffle=False)  

noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2')
#noise_scheduler = DDIMScheduler.from_pretrained("google/ddpm-cifar10-32")
#noise_scheduler.set_timesteps(num_inference_steps=50)
#noise_scheduler.config.clip_sample = False
net = UNet2DModel().to("cuda")
eval_model = evaluation_model()
denormalize = transforms.Compose([
    transforms.Normalize(mean=(-1, -1, -1), std=(2, 2, 2))  
])
#checkpoint = torch.load("/kaggle/working/weights/best.ckpt")
weight_path = "/kaggle/input/lab6-best-weight/best.ckpt"
net = torch.load(weight_path)
decode_result = sample_and_save(net,noise_scheduler,new_test_loader)
data_iterator = iter(new_test_loader)
batch = next(data_iterator)
_, labels = batch
acc = eval_model.eval(decode_result,labels)
print(f'New Test acc is {acc}')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:00<00:00, 120.02s/it]


New Test acc is 0.7638888888888888
